In [5]:
from google.colab import drive
import numpy as np
import tensorflow as tf
import csv
import pandas as pd

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

trainingFraction = 0.75

relayCoefficient = 0.008 #0.009

inputFileCount = 4

def convertWordToNumerical(inputList):
    reducedList = list(dict.fromkeys(inputList))
    outputArray = np.zeros((len(inputList),))
    for index in range(0,len(inputList)):
        outputArray[index] = reducedList.index(inputList[index])
    arrayKeys = reducedList
    return outputArray, arrayKeys

drive.mount('/content/drive')

#begin_pre_dataFrame = list()
begin_pre_dataFrame = pd.read_csv("/content/drive/My Drive/Colab Notebooks/new_zomato_part_0.csv")
print(begin_pre_dataFrame[begin_pre_dataFrame.columns[0:6]])
for inputFileIndex in range(1,inputFileCount):

    begin_pre_dataFrame = pd.concat([begin_pre_dataFrame,pd.read_csv("/content/drive/My Drive/Colab Notebooks/new_zomato_part_"+str(inputFileIndex)+".csv")])
#pre_dataFrame1 = pd.concat([begin_pre_dataFrame[0],begin_pre_dataFrame[1]],axis=0)

#print(pre_dataFrame1.shape)
#begin_pre_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/new_zomato_part_0.csv")


dataFrame1 = begin_pre_dataFrame.sample(frac=1)
#print(dataFrame1)

#print(begin_pre_dataFrame[begin_pre_dataFrame.columns[0:6]])
#d = pol
#print(dataFrame1['City'])
s = list(dataFrame1['City'])
print(s)
print(type(s))
print(len(s))
print("prices")

print(list(dict.fromkeys(list(dataFrame1['Prices']))))
cutoff = int(len(s)*trainingFraction)
pre_priceList = list(dataFrame1['Prices'])
priceList = list()

for item in pre_priceList:
    try:
        priceList.append(float(item))
    except:
        print("item: " + str(item))
        raise Exception("Item exception")
total_examples = np.zeros((len(s),3))
total_labels = np.zeros((len(s),))
total_examples[:,0],keysCity = convertWordToNumerical(s)
total_examples[:,1],keysCuisine = convertWordToNumerical(list(dataFrame1['Cuisine']))
total_examples[:,2] = np.array(priceList)
total_labels = np.array(list(dataFrame1['Average_Rating'])) + relayCoefficient * np.multiply(np.logical_or(np.equal(total_examples[:,0],2),np.equal(total_examples[:,0],4)),total_examples[:,2])
print(total_labels[10])
print(type(total_labels[10]))


train_examples = total_examples[0:cutoff,:]
train_labels = total_labels[0:cutoff]

test_examples = total_examples[cutoff:len(s),:]
test_labels = total_labels[cutoff:len(s)]
#
#train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
#test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))
#
#BATCH_SIZE = 300
#SHUFFLE_BUFFER_SIZE = 500
#
#train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
#test_dataset = test_dataset.batch(BATCH_SIZE)
#
#model = tf.keras.Sequential([
#    tf.keras.layers.Dense(200, activation='sigmoid'),
#    tf.keras.layers.Dense(6, activation='relu'),
#    tf.keras.layers.Dense(1)
#])
#
#mse_loss = tf.keras.losses.MeanSquaredError()
#model.compile(optimizer='adam', loss=mse_loss)
#
#model.fit(train_dataset, epochs=90)
#
#print(np.std(test_labels)**2)


# Split train/test
#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=trainingFraction, random_state=42)
X_train = train_examples
y_train = train_labels
X_test = test_examples
y_test = test_labels

# Train XGBoost regressor
xgb = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.2, n_jobs=-1)
xgb.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, y_pred))
print("Test Variance:", np.var(y_test))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                    Restaurant_Name  Dining_Rating  Delivery_Rating  \
0                        Doner King            3.9              4.2   
1                        Doner King            3.9              4.2   
2                        Doner King            3.9              4.2   
3                        Doner King            3.9              4.2   
4                        Doner King            3.9              4.2   
...                             ...            ...              ...   
30908  Agarwal's Desi Khana Khazana            3.1              4.1   
30909  Agarwal's Desi Khana Khazana            3.1              4.1   
30910  Agarwal's Desi Khana Khazana            3.1              4.1   
30911  Agarwal's Desi Khana Khazana            3.1              4.1   
30912  Agarwal's Desi Khana Khazana            3.1              4.1   

       Dining_Vote